In [64]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')

import os
os.environ["OPENAI_API_KEY"] = config['API']['OPENAI_API_KEY']

from deepeval.metrics import *
from deepeval.test_case import *

from deepeval.benchmarks import MMLU, HellaSwag
from deepeval.benchmarks.tasks import MMLUTask, HellaSwagTask

from langchain_community.llms import GPT4All
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.output_parsers import StrOutputParser

input = "The dog chased the cat up the tree, who ran up the tree?"
expected_output = "cat."
local_path = './models/nous-hermes-llama2-13b.Q4_0.gguf'


model = ChatOpenAI(
    verbose=True,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=0.7,
)
llm_chain = (                     
    PromptTemplate(input_variables=["question"], template=input)
    | model 
    | StrOutputParser()
)
actual_output = llm_chain.invoke({"question": ""})

# model = GPT4All(
#     model=local_path,
#     # callbacks=[ChainStreamHandler(g)],
#     streaming=True,
#     verbose=True,
# )
# llm_chain = PromptTemplate(input_variables=["text"], template=input) | model
# actual_output = llm_chain.invoke({"text": ""})
# _res

The cat ran up the tree.

In [31]:
mmlu_benchmark = MMLU().load_benchmark_dataset(MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE)
data = [{'input': mmlu.input, 'expected_output': mmlu.expected_output} for mmlu in mmlu_benchmark]
data[:3], len(data)

([{'input': 'Let x = 1. What is x << 3 in Python 3?\nA. 1\nB. 3\nC. 8\nD. 16\nAnswer:',
   'expected_output': 'C'},
  {'input': 'In Python 3, which of the following function convert a string to an int in python?\nA. int(x [,base])\nB. long(x [,base] )\nC. float(x)\nD. str(x)\nAnswer:',
   'expected_output': 'A'},
  {'input': "A user enters a Web address in a browser, and a request for a file is sent to a Web server. Which of the following best describes how the file is sent to the user?\nA. The file is broken into packets for transmission. The packets must be reassembled upon receipt.\nB. The file is broken into packets for transmission. The user's browser must request each packet in order until all packets are received.\nC. The server attempts to connect directly to the user's computer. If the connection is successful, the entire file is sent. If the connection is unsuccessful, an error message is sent to the user.\nD. The server repeatedly attempts to connect directly to the user's c

In [29]:
# Define benchmark with specific tasks and shots
# benchmark = MMLU()
hella_benchmark = HellaSwag().load_benchmark_dataset(HellaSwagTask.TRIMMING_BRANCHES_OR_HEDGES)
data = [{'input': hella.input, 'expected_output': hella.expected_output} for hella in hella_benchmark]
data[:3], len(data)

([{'input': 'A man is shown working in an outdoor garden. he\nA. is using tools as he works.\nB. uses a large tool to bark out a few new leaves.\nC. uses trowels and strikes a tree.\nD. uses a sponge and a trowel to scrub down the side of a tree.\nAnswer:',
   'expected_output': 'A'},
  {'input': 'A man is standing outside turning on his hedge trimmer turning it around to make the blades go. He then walks over to the hedges and begins to cut down the leaves on it. as he\nA. cuts it asia walks towards the lawn.\nB. is cutting by himself, a car drives by that is parked over in the yard.\nC. is cutting it, he begins to walk into the hedges to cut the hedge evenly and shaking the cut leaves off with his hand.\nD. is doing his mowing he trips and falls down.\nAnswer:',
   'expected_output': 'C'},
  {'input': 'A man has climbed a large ladder outside. he\nA. is using trimmers to cut and trim large trees.\nB. ropes and lures a cow into the open.\nC. is using it to pull himself up onto a platf

In [46]:
metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the expected output.",
    # NOTE: you can only provide either criteria or evaluation_steps, and not both
    evaluation_steps=[
        "Check whether the facts in 'actual output' indicates the given 'expected output'",
        "It does not matter whether the 'actual output' has full context or not.",
        "If indicating to correct 'expected output' answer must be OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
) 

template = """
    You are a wise and precise chatbot.
    With the below given questions, you will have to solve the question and answer the question instructed with several options to select
    {question}
    Provide only the letter corresponding to your answer (e.g., "A", "B", "C", etc.).
"""

llm_chain = (                     
    PromptTemplate(input_variables=["question"], template=template)
    | model 
    | StrOutputParser()
)

test_result = []
for evaluate in data:
    # actual_output = llm_chain.invoke({"question": "It depends, some might consider the cat, while others might argue the dog."})
    actual_output = llm_chain.invoke({"question": evaluate['input']})
    # actual_output = "A"

    test_case = LLMTestCase(
        input=evaluate['input'],
        actual_output=actual_output,
        expected_output=evaluate['expected_output']
    )

    metric.measure(test_case)
    result = {'metric_score' : metric.score, 'metric_reason': metric.reason}
    test_result.append(result)

test_result

D. 16

c:\Users\whdgh\Desktop\projects\Langchain_Embedding\.venv\lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Event loop is already running. Applying nest_asyncio patch to allow async execution...

c:\Users\whdgh\Desktop\projects\Langchain_Embedding\.venv\lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

A. int(x [,base])

...

C.

c:\Users\whdgh\Desktop\projects\Langchain_Embedding\.venv\lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Event loop is already running. Applying nest_asyncio patch to allow async execution...

c:\Users\whdgh\Desktop\projects\Langchain_Embedding\.venv\lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[{'metric_score': 1.0000000000000002,
  'metric_reason': 'The actual output correctly identifies that x << 3 results in 16, as indicated by option D.'},
 {'metric_score': 1.0,
  'metric_reason': "The actual output correctly identifies 'int(x [,base])' which matches the expected answer."},
 {'metric_score': 1.0,
  'metric_reason': 'The actual output correctly identifies that the file is broken into packets for transmission and needs to be reassembled upon receipt, which matches the expected answer.'},
 {'metric_score': 0.0,
  'metric_reason': "The Actual Output 'A' does not indicate a lossless transformation, as compressing the image may lose information."},
 {'metric_score': 1.0,
  'metric_reason': "The actual output correctly identifies 'B. How many programming statements the program contains' as the least likely consideration affecting the program's ability to process larger data sets."},
 {'metric_score': 0.030572072589472032,
  'metric_reason': "The actual output 'D. 256' is incorr

In [51]:
import statistics

len(test_result), test_result
avg = statistics.mean([eval['metric_score'] for eval in test_result])
print(f"모델: ChatOpenAI\nGEval 평가 평균: {avg}")

모델: ChatOpenAI
GEval 평가 평균: 0.7908556096172605
